[참고](https://github.com/rickiepark/handson-ml2/blob/master/16_nlp_with_rnns_and_attention.ipynb)

- 챗봇(chatbot) : 대화의 상대방이 자신을 사람이라고 생각하도록 속일 수 있는 기계.
- 문자 단위 RNN(character RNN) : 문장에서 다음 글자를 예측하도록하는 훈련
- **상태가 없는 RNN**(stateless RNN)을 사용하고 **상태가 있는 RNN**(stateful RNN)을 구축한다.
- 텐서플로 애드온(Addon) 프로젝트에서 제공하는 seq2seq API를 사용한다.
- **트랜스포머**(transformer) : RNN을 모두 제거하고 어텐션만 사용해 매우 좋은 성능을 내는 구조. GPT-2와 BERT 같은 모델의 기반이 된다.

# 16.1 Char-RNN을 사용해 셰익스피어 문체 생성하기
## 16.1.1 훈련 데이터셋 만들기

In [2]:
# 셰익스피어 작품 다운로드
import tensorflow as tf
from tensorflow import keras

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

1122304/1115394 [==============================] - 1s 1us/step


In [5]:
# 글자를 정수로 인코딩
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True) # 글자 수준 인코딩
tokenizer.fit_on_texts(shakespeare_text) 

In [6]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [7]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [8]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

In [10]:
# 전체 텍스트를 인코딩하여 각 글자를 ID로 나타냄
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1 # 1에서 39 대신 0에서 38까지 ID를 얻기 위해 1을 뺌

## 16.1.2 순차 데이터셋을 나누는 방법
훈련 세트, 검증 세트, 테스트 세트가 중복되지 않도록 만드는 것이 중요. 시계열을 다룰 때는 보통 시간에 따라 나눈다. <br/>
암묵적으로 RNN은 시계열 데이터가 넓은 의미에서 **변하지 않는다**(stationary)고 가정한다.

In [11]:
# 텍스트 처음 90%를 훈련 세트로 사용
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

## 16.1.3 순차 데이터를 윈도 여러 개로 자르기
`window()` 메서드를 사용해 긴 시퀀스를 작은 많은 텍스트 윈도로 변환한다. <br/>
- TBPTT(truncated backpropagation through time) : 부분 문자열 길이만큼만 역전파를 위해 펼쳐지는 것.

In [12]:
n_steps = 100 # n_steps를 너무 짧게 하면 안 됨.
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True) # 모든 윈도가 동일한 글자를 포함하도록 지정.

- 중첩 데이터셋(nested dataset) : 하나의 데이터셋으로 표현되는 윈도를 포함하는 데이터셋. 섞거나 배치를 만들 때 유용.
- 플랫 데이터셋(flat dataset) : 데이터셋이 들어 있지 않는 데이터셋. 모델에 데이터셋이 아니라 텐서를 넣어야 하기 때문에 중첩 데이터셋 대신 사용.
- ex) {{1, 2}, {3, 4, 5}} (중첩 데이터셋) ---(`flat_map` 메서드)---> {1, 2, 3, 4, 5} (플랫 데이터셋)

In [13]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [14]:
# 윈도를 배치로 만들고 입력과 타깃을 분리
np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [15]:
# 원-핫 벡터를 사용해 글자를 인코딩
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

In [16]:
# 프리페칭 추가
dataset = dataset.prefetch(1)

## 16.1.4 Char-RNN 모델 만들고 훈련하기

In [ ]:
model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                     dropout=0.2), #recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10)

Epoch 1/10
 9426/31370 [========>.....................] - ETA: 46:51 - loss: 1.8856

## 16.1.5 Char-RNN 모델 사용하기

In [ ]:
# 전처리를 위한 함수
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [ ]:
# 어떤 텍스트의 다음 글자 예측
X_new = preprocess(["How are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

## 16.1.6 가짜 셰익스피어 텍스트를 생성하기
생성된 텍스트의 다양성을 제어하려면 **온도**(temperature)라고 불리는 숫자로 로짓을 나눈다. 0에 가까울수록 높은 확률을 가진 글자를 택한다.

In [ ]:
tf.random.set_seed(42)

tf.random.categorical([[np.log(0.5), np.log(0.4), np.log(0.1)]], num_samples=40).numpy()

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
# 함수를 반복 호출하여 다음 글자를 얻고 텍스트에 추가하는 함수
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [ ]:
# 온도를 다르게 해보며 테스트
tf.random.set_seed(42)

print(complete_text("t", temperature=0.2))

In [ ]:
print(complete_text("t", temperature=1)) # 이 셰익스피어 모델은 1에 가까운 온도에서 잘 작동하는 듯.

In [ ]:
print(complete_text("t", temperature=2))